In [42]:
import requests
import orjson

In [43]:
AC_URL = "https://answercoalesce.renci.org/query"

In [65]:
in_message1 = {
        "parameters": {
            "pvalue_threshold": 1e-5,
            "result_length": 100,
            "predicates_to_exclude": [
                "biolink:causes", "biolink:biomarker_for", "biolink:contraindicated_for", "biolink:contraindicated_in",
                "biolink:contributes_to", "biolink:has_adverse_event", "biolink:causes_adverse_event",
                "biolink:treats_or_applied_or_studied_to_treat"
            ]
        },
        "message": {
            "query_graph": {
                "nodes": {
                    "drug": {
                        "categories": [
                            "biolink:Drug"
                        ],
                        "is_set": False,
                        "constraints": []
                    },
                    "disease": {
                        "ids": [
                            "MONDO:0004975"
                        ],
                        "is_set": False,
                        "categories": [
                            "biolink:Disease"
                        ],
                        "constraints": []
                    }
                },
                "edges": {
                    "e00": {
                        "subject": "drug",
                        "object": "disease",
                        "predicates": [
                            "biolink:treats"
                        ],
                        "knowledge_type": "inferred",
                        "attribute_constraints": [],
                        "qualifier_constraints": []
                    }
                }
            }
        }
    }


In [37]:
in_message1["message"]["query_graph"]

{'nodes': {'drug': {'categories': ['biolink:Drug'],
   'is_set': False,
   'constraints': []},
  'disease': {'ids': ['MONDO:0004975'], 'is_set': False, 'constraints': []}},
 'edges': {'e00': {'subject': 'drug',
   'object': 'disease',
   'predicates': ['biolink:treats'],
   'knowledge_type': 'inferred',
   'attribute_constraints': [],
   'qualifier_constraints': []}}}

In [68]:
def drug_to_disease_query(message):
    response = requests.post( AC_URL, json = message )
    print(response.status_code)
    return response.json()

answerset = drug_to_disease_query(in_message1)


200


In [70]:
import json
from string import Template

def qg_template():
    return '''{
        "query_graph": {
            "nodes": {
                "$source": {
                    "ids": $source_id,
                    "constraints": [],
                    "is_set": false,
                    "categories": $source_category
                },
                "$target": {
                    "ids": $target_id,
                    "is_set": false,
                    "constraints": [],
                    "categories": $target_category
                }
            },
            "edges": {
                "e00": {
                    "subject": "$source",
                    "object": "$target",
                    "predicates": $predicate,
                    "attribute_constraints": [],
                    "qualifier_constraints": $qualifier
                }
            }
        }
    }'''

def get_qg(curie, is_source, predicates, source_category='', target_category='', object_aspect_qualifier=None, object_direction_qualifier=None):
    if not is_source:
        target_ids = curie
        source_ids = []
    else:
        source_ids = curie
        target_ids = []

    source = source_category.split(":")[1].lower() if source_category else 'n0'
    target = target_category.split(":")[1].lower() if target_category else 'n1'
    query_template = Template(qg_template())
    query = {}

    source_category = [source_category] if source_category else []
    target_category = [target_category] if target_category else []

    quali = []
    if object_aspect_qualifier and object_direction_qualifier:
        quali = [
            {
                "qualifier_set": [
                    {
                        "qualifier_type_id": "biolink:object_aspect_qualifier",
                        "qualifier_value": object_aspect_qualifier
                    },
                    {
                        "qualifier_type_id": "biolink:object_direction_qualifier",
                        "qualifier_value": object_direction_qualifier
                    }
                ]
            }
        ]

    qs = query_template.substitute(source=source, target=target, source_id=json.dumps(source_ids),
                                   target_id=json.dumps(target_ids),
                                   source_category=json.dumps(source_category),
                                   target_category=json.dumps(target_category), predicate=json.dumps(predicates),
                                   qualifier=json.dumps(quali))

    try:
        query = json.loads(qs)
        if not is_source:
            del query["query_graph"]["nodes"][source]["ids"]
        else:
            del query["query_graph"]["nodes"][target]["ids"]
    except UnicodeDecodeError as e:
        print(e)
    return query

parameters = {
    "pvalue_threshold": 1e-5,
    "result_length": 100,
    "predicates_to_exclude": [
        "biolink:causes", "biolink:biomarker_for", "biolink:contraindicated_for", "biolink:contraindicated_in",
        "biolink:contributes_to", "biolink:has_adverse_event", "biolink:causes_adverse_event",
        "biolink:treats_or_applied_or_studied_to_treat"
    ]
}

message = get_qg(
    curie=["MONDO:0004975"],
    is_source=False,
    predicates=["biolink:treats"],
    source_category="biolink:Drug",
    target_category="biolink:Disease",
    object_aspect_qualifier=None,
    object_direction_qualifier=None
)

final_query = {
    "parameters": parameters,
    "message": message
}

print(json.dumps(final_query, indent=2))
# answerset1 = drug_to_disease_query(final_query)

{
  "parameters": {
    "pvalue_threshold": 1e-05,
    "result_length": 100,
    "predicates_to_exclude": [
      "biolink:causes",
      "biolink:biomarker_for",
      "biolink:contraindicated_for",
      "biolink:contraindicated_in",
      "biolink:contributes_to",
      "biolink:has_adverse_event",
      "biolink:causes_adverse_event",
      "biolink:treats_or_applied_or_studied_to_treat"
    ]
  },
  "message": {
    "query_graph": {
      "nodes": {
        "drug": {
          "constraints": [],
          "is_set": false,
          "categories": [
            "biolink:Drug"
          ]
        },
        "disease": {
          "ids": [
            "MONDO:0004975"
          ],
          "is_set": false,
          "constraints": [],
          "categories": [
            "biolink:Disease"
          ]
        }
      },
      "edges": {
        "e00": {
          "subject": "drug",
          "object": "disease",
          "predicates": [
            "biolink:treats"
          ],
     

In [67]:
query_graph = answerset["message"]["query_graph"]
kg_edges = answerset["message"]["knowledge_graph"]["edges"]
kg_nodes = answerset["message"]["knowledge_graph"]["nodes"]
results = answerset["message"]["results"]
aux_graphs = answerset["message"]["auxiliary_graphs"]

for qedge_id, qedges  in query_graph.get("edges", {}).items():
    subject = query_graph.get("nodes", {})[qedges["subject"]]
    object = query_graph.get("nodes", {})[qedges["object"]]
    if subject.get("ids", []):
        is_source = True
    else:
        is_source = False


# Let's trace one result

In [47]:
len(results)

1246

In [1]:
results

NameError: name 'results' is not defined

In [48]:
# support_graphs = None
# for i in range(len(results)):
#     for eid, edge in results[i]["analyses"][0]["edge_bindings"].items():
#         inferred_edge = edge[0]["id"]
#         inferred_edge_string = f'{kg_nodes[kg_edges[inferred_edge]["subject"]]["name"]}-({kg_edges[inferred_edge]["predicate"]})->{kg_nodes[kg_edges[inferred_edge]["object"]]["name"]}'
#         #Let's see the inferred_edge in the KG then extract it support graphs
#         # support_graphs = [attributes["value"] for attributes in kg_edges[inferred_edge]["attributes"] if attributes["attribute_type_id"] == "biolink:support_graphs"]
#         # if len(support_graphs)==1: 
#         print(f'{i+1}: {inferred_edge_string}')
#                 # print(i)
#                 # break


In [49]:
# Now let's explore the path for any of the results
i = 1

support_graphs = None
for eid, edge in results[i]["analyses"][0]["edge_bindings"].items():
    inferred_edge = edge[0]["id"]
    #Let's see the inferred_edge in the KG then extract it support graphs
    support_graphs = [attributes["value"] for attributes in kg_edges[inferred_edge]["attributes"] if attributes["attribute_type_id"] == "biolink:support_graphs"]
    support_graphs = sorted(support_graphs)
    # if len(support_graphs) > 1: 
    print(f'{inferred_edge}==> \n{" ^ \n".join(support_graphs)}')
        # print(i)
        # break


CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975==> 
e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:43 ^ 
e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:590 ^ 
e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:65036 ^ 
e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:83817 ^ 
e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_UniProtKB:P81908 ^ 
n_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:has_chemical_role_CHEBI_ROLE_EC_3.1.1_carboxylic_ester_hydrolase_inhibitor ^ 
n_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:has_chemical_role_CHEBI_ROLE_neurotransmitter_agent


In [51]:
g_support_graph = support_graphs[0] # Graph(edge) Inferred
p_support_graph = support_graphs[-1] # Property(node) Inferred

#### This implies that the inference was driven by both graph enrichment and property enrichment

### 1. GRAPH ENRICHMENT ROUTE: e_Inferred_SG:........'


In [162]:
# Tracing the support graph to the auxiliary components
aux_graph_edges = aux_graphs.get(g_support_graph).get("edges")
print(f'{inferred_edge}---> {" ^ ".join(aux_graph_edges)}')

# # The aux_graph has 3 edges
# 1. Inferred_node to the enrichment
# 2. Enrichment to the Groupset enriched
# 3. the Groupset enriched to the qg Curie

CHEBI:28939_Inferred_to_biolink:treats_MONDO:0004979---> CHEBI:28939_biolink:treats_MONDO:0003781 ^ e_uuid:1_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781 ^ uuid:1_biolink:treats_MONDO:0004979


In [163]:
aux_graph_edges

['CHEBI:28939_biolink:treats_MONDO:0003781',
 'e_uuid:1_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
 'uuid:1_biolink:treats_MONDO:0004979']

##### Let's dig further down the path

In [164]:
inf2enrichment_aux_graph_edge = ''
enrich2group_aux_graph_edge = ''
group2curie_aux_graph_edge = ''
for aedge in aux_graph_edges:
    support = [attributes["value"] for attributes in kg_edges[aedge]["attributes"]]
    if support:
        if isinstance(support[0], list):
            enrich2group_aux_graph_edge = aedge
        else:
            group2curie_aux_graph_edge = aedge
    else:
        inf2enrichment_aux_graph_edge = aedge

In [165]:
# The first edge: 
inf2enrichmentedge=kg_edges[inf2enrichment_aux_graph_edge]
inf2enrichmentedge

{'subject': 'CHEBI:28939',
 'object': 'MONDO:0003781',
 'predicate': 'biolink:treats',
 'sources': [{'resource_id': 'infores:drugcentral',
   'resource_role': 'primary_knowledge_source'}],
 'attributes': []}

In [166]:
print(f"{kg_nodes[inf2enrichmentedge['subject']]['name']} -({inf2enrichmentedge['predicate']})-> {kg_nodes[inf2enrichmentedge['object']]['name']}")
print()
print(inf2enrichmentedge)
# implies:  inferred_node -(rel)- enrichment. It has no support graph so we stop digging

N-acetyl-L-cysteine -(biolink:treats)-> bronchitis

{'subject': 'CHEBI:28939', 'object': 'MONDO:0003781', 'predicate': 'biolink:treats', 'sources': [{'resource_id': 'infores:drugcentral', 'resource_role': 'primary_knowledge_source'}], 'attributes': []}


In [167]:
# The second edge: 
enrichment2group_edge = kg_edges[enrich2group_aux_graph_edge]
enrichment2group_edge
# This is the enrichment - (rel) - lookup_set 

{'subject': 'uuid:1',
 'object': 'MONDO:0003781',
 'predicate': 'biolink:treats_or_applied_or_studied_to_treat',
 'sources': [{'resource_id': 'infores:answercoalesce',
   'resource_role': 'primary_knowledge_source'}],
 'qualifiers': [],
 'attributes': [{'attribute_type_id': 'biolink:support_graphs',
   'value': ['SG:_e_CHEBI:8378_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
    'SG:_e_CHEBI:3897_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
    'SG:_e_CHEBI:9449_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
    'SG:_e_CHEBI:91495_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
    'SG:_e_CHEBI:28177_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
    'SG:_e_CHEBI:18357_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
    'SG:_e_CHEBI:8245_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
    'SG:_e_CHEBI:9667_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781',
    'SG:_e_CHEB

In [168]:
# The enrichment - groupset has many support graph, so we dig further
print(f"{enrich2group_aux_graph_edge}:===>")
print()
for attributes in enrichment2group_edge["attributes"]:
    enrichment2group_support_graphs = attributes["value"]
    # Each of these exists in the auxiliary graph
    for i, e2group_sp in enumerate(enrichment2group_support_graphs):
        e2group_edges = aux_graphs[e2group_sp]["edges"]
        theedges0 = kg_edges[e2group_edges[0]]
        theedges1 = kg_edges[e2group_edges[1]]
        print(f'Path{i+1}: {kg_nodes[theedges0["subject"]]["name"]}--({theedges0["predicate"]})-->{kg_nodes[theedges0["object"]]["name"]} ^ {kg_nodes[theedges1["subject"]]["name"]}--({theedges1["predicate"]})-->{kg_nodes[theedges1["object"]]["name"]}')
        print()
# 4 members of set uuid affects `NCBIGene:83817`         

e_uuid:1_biolink:treats_or_applied_or_studied_to_treat_MONDO:0003781:===>

Path1: prednisolone--(biolink:member_of)-->uuid:1 ^ prednisolone--(biolink:treats_or_applied_or_studied_to_treat)-->bronchitis

Path2: Cortisone acetate--(biolink:treats_or_applied_or_studied_to_treat)-->bronchitis ^ Cortisone acetate--(biolink:member_of)-->uuid:1

Path3: terbutaline--(biolink:treats_or_applied_or_studied_to_treat)-->bronchitis ^ terbutaline--(biolink:member_of)-->uuid:1

Path4: 3-hydroxy-2-phenylpropanoic acid (8-methyl-8-propan-2-yl-8-azoniabicyclo[3.2.1]octan-3-yl) ester--(biolink:treats_or_applied_or_studied_to_treat)-->bronchitis ^ 3-hydroxy-2-phenylpropanoic acid (8-methyl-8-propan-2-yl-8-azoniabicyclo[3.2.1]octan-3-yl) ester--(biolink:member_of)-->uuid:1

Path5: theophylline--(biolink:treats_or_applied_or_studied_to_treat)-->bronchitis ^ theophylline--(biolink:member_of)-->uuid:1

Path6: (R)-noradrenaline--(biolink:member_of)-->uuid:1 ^ (R)-noradrenaline--(biolink:treats_or_applied_or_stu

In [135]:
aux_graphs[e2group_sp]["edges"]

['CHEBI:5783_member_of_uuid:1', 'e_CHEBI:5783_biolink:affects_NCBIGene:1437']

In [136]:
# The third edge: 
print(kg_edges[group2curie_aux_graph_edge])

# implies:  the Groupset- qg Curie edge. It has support graph of members of the group, so we stop digging

{'subject': 'uuid:1', 'object': 'MONDO:0004979', 'predicate': 'biolink:treats', 'sources': [{'resource_id': 'infores:answercoalesce', 'resource_role': 'primary_knowledge_source'}], 'attributes': [{'attribute_type_id': 'biolink:support_graphs', 'value': 'SG:_uuid:1_biolink:treats_MONDO:0004979', 'attribute_source': 'infores:answercoalesce'}]}


In [137]:
group2curie_aux_graph_edge

'uuid:1_biolink:treats_MONDO:0004979'

In [138]:
# CURIE-PHYSIO
print(f"{group2curie_aux_graph_edge}:===>")
for attributes in kg_edges[group2curie_aux_graph_edge]["attributes"]:
    sgs = attributes['value']
    sgedges = sorted(aux_graphs[sgs]["edges"])
    j = 0; path = 1
    while j < len(sgedges)-1:
        p1 = kg_edges[sgedges[j]]
        p2 = kg_edges[sgedges[j+1]]
        print(f'Path{path}: {kg_nodes[p2["subject"]]["name"]}-({p2["predicate"]})->{kg_nodes[p2["object"]]["name"]} ^ {kg_nodes[p1["subject"]]["name"]}-({p1["predicate"]})->{kg_nodes[p1["object"]]["name"]}')
        j+=2
        path+=1

uuid:1_biolink:treats_MONDO:0004979:===>
Path1: zafirlukast-(biolink:member_of)->uuid:1 ^ zafirlukast-(biolink:treats)->asthma
Path2: zileuton-(biolink:member_of)->uuid:1 ^ zileuton-(biolink:treats)->asthma
Path3: pemirolast-(biolink:member_of)->uuid:1 ^ pemirolast-(biolink:treats)->asthma
Path4: (-)-ephedrine-(biolink:member_of)->uuid:1 ^ (-)-ephedrine-(biolink:treats)->asthma
Path5: (R)-noradrenaline-(biolink:member_of)->uuid:1 ^ (R)-noradrenaline-(biolink:treats)->asthma
Path6: desonide-(biolink:member_of)->uuid:1 ^ desonide-(biolink:treats)->asthma
Path7: albuterol-(biolink:member_of)->uuid:1 ^ albuterol-(biolink:treats)->asthma
Path8: theophylline-(biolink:member_of)->uuid:1 ^ theophylline-(biolink:treats)->asthma
Path9: (R)-adrenaline-(biolink:member_of)->uuid:1 ^ (R)-adrenaline-(biolink:treats)->asthma
Path10: beclomethasone-(biolink:member_of)->uuid:1 ^ beclomethasone-(biolink:treats)->asthma
Path11: betamethasone-(biolink:member_of)->uuid:1 ^ betamethasone-(biolink:treats)->as

In [139]:
# PROPERTY ENRICHED SG
p_support_graph = support_graphs[-1] # Property Inferred

In [140]:
# Tracing the support graph to the auxiliary components
aux_graph_edges = aux_graphs.get(p_support_graph).get("edges")
print(f'{inferred_edge}---> {" ^ ".join(aux_graph_edges)}')

# # The aux_graph has 3 edges
# 1. Inferred_node to the enrichment
# 2. Enrichment to the Groupset enriched
# 3. the Groupset enriched to the qg Curie

CHEBI:16480_Inferred_to_biolink:treats_MONDO:0004979---> e_CHEBI_ROLE_bronchodilator_agent_biolink:similar_to_uuid:1 ^ uuid:1_biolink:treats_MONDO:0004979 ^ CHEBI:16480_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent


In [141]:
inf2enrichment_aux_graph_edge = ''
enrich2group_aux_graph_edge = ''
group2curie_aux_graph_edge = ''
for aedge in aux_graph_edges:
    support = [attributes["value"] for attributes in kg_edges[aedge]["attributes"]]
    if support:
        if isinstance(support[0], list):
            enrich2group_aux_graph_edge = aedge
        else:
            group2curie_aux_graph_edge = aedge
    else:
        inf2enrichment_aux_graph_edge = aedge

##### Let's dig further down the path

In [142]:
kg_edges[inf2enrichment_aux_graph_edge]


{'subject': 'CHEBI:16480',
 'object': 'CHEBI_ROLE_bronchodilator_agent',
 'predicate': 'biolink:has_chemical_role',
 'sources': [{'resource_id': 'infores:answercoalesce',
   'resource_role': 'primary_knowledge_source'}],
 'attributes': []}

In [143]:
# The first edge: 
inf2enrichmentedges=kg_edges[inf2enrichment_aux_graph_edge]

print(f"{kg_nodes[inf2enrichmentedges['subject']]['name']} -({inf2enrichmentedges['predicate']})-> {kg_nodes[inf2enrichmentedges['object']]['name']}")
print()
print(inf2enrichmentedges)
# implies:  inferred_node -(rel)- enrichment. It has no support graph so we stop digging

nitric oxide -(biolink:has_chemical_role)-> CHEBI_ROLE_bronchodilator_agent

{'subject': 'CHEBI:16480', 'object': 'CHEBI_ROLE_bronchodilator_agent', 'predicate': 'biolink:has_chemical_role', 'sources': [{'resource_id': 'infores:answercoalesce', 'resource_role': 'primary_knowledge_source'}], 'attributes': []}


In [144]:
# The second edge: 
enrichment2group_edge = kg_edges[enrich2group_aux_graph_edge]
enrichment2group_edge
# This is the enrichment - (rel) - lookup_set 

{'subject': 'CHEBI_ROLE_bronchodilator_agent',
 'object': 'uuid:1',
 'predicate': 'biolink:similar_to',
 'sources': [{'resource_id': 'infores:answercoalesce',
   'resource_role': 'primary_knowledge_source'}],
 'qualifiers': [],
 'attributes': [{'attribute_type_id': 'biolink:support_graphs',
   'value': ['SG:_n_CHEBI:4728_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent',
    'SG:_n_PUBCHEM.COMPOUND:5487426_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent',
    'SG:_n_CHEBI:64317_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent',
    'SG:_n_CHEBI:28177_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent',
    'SG:_n_CHEBI:3207_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent',
    'SG:_n_CHEBI:6792_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent',
    'SG:_n_CHEBI:408174_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent',
    'SG:_n_CHEBI:9449_biolink:has_chemical_role_CHEBI_ROLE_bronchodilator_agent',
    'SG:_n_PUBCHEM.COMPOU

In [145]:
# The enrichment - groupset has many support graph, so we dig further
print(f"{enrich2group_aux_graph_edge}:===>")
for attributes in enrichment2group_edge["attributes"]:
    enrichment2group_support_graphs = attributes["value"]
    
    # Each of these exists in the auxiliary graph
    for i, e2group_sp in enumerate(enrichment2group_support_graphs):
        e2group_edges = aux_graphs[e2group_sp]["edges"]
        theedges0 = kg_edges[e2group_edges[0]]
        theedges1 = kg_edges[e2group_edges[1]]
        print(f'Path{i+1}: {kg_nodes[theedges0["subject"]]["name"]}--({theedges0["predicate"]})-->{kg_nodes[theedges0["object"]]["name"]} ^ {kg_nodes[theedges1["subject"]]["name"]}--({theedges1["predicate"]})-->{kg_nodes[theedges1["object"]]["name"]}')

# 4 members of set uuid affects `NCBIGene:83817`         

e_CHEBI_ROLE_bronchodilator_agent_biolink:similar_to_uuid:1:===>
Path1: dyphylline--(biolink:has_chemical_role)-->CHEBI_ROLE_bronchodilator_agent ^ dyphylline--(biolink:member_of)-->uuid:1
Path2: Tiotropium bromide--(biolink:has_chemical_role)-->CHEBI_ROLE_bronchodilator_agent ^ Tiotropium bromide--(biolink:member_of)-->uuid:1
Path3: isoprenaline--(biolink:member_of)-->uuid:1 ^ isoprenaline--(biolink:has_chemical_role)-->CHEBI_ROLE_bronchodilator_agent
Path4: theophylline--(biolink:has_chemical_role)-->CHEBI_ROLE_bronchodilator_agent ^ theophylline--(biolink:member_of)-->uuid:1
Path5: budesonide--(biolink:has_chemical_role)-->CHEBI_ROLE_bronchodilator_agent ^ budesonide--(biolink:member_of)-->uuid:1
Path6: metaproterenol--(biolink:has_chemical_role)-->CHEBI_ROLE_bronchodilator_agent ^ metaproterenol--(biolink:member_of)-->uuid:1
Path7: arformoterol--(biolink:member_of)-->uuid:1 ^ arformoterol--(biolink:has_chemical_role)-->CHEBI_ROLE_bronchodilator_agent
Path8: terbutaline--(biolink:ha

In [146]:
# The third edge: 
kg_edges[group2curie_aux_graph_edge]
# implies:  the Groupset- qg Curie edge. It has support graph of members of the group, so we stop digging

{'subject': 'uuid:1',
 'object': 'MONDO:0004979',
 'predicate': 'biolink:treats',
 'sources': [{'resource_id': 'infores:answercoalesce',
   'resource_role': 'primary_knowledge_source'}],
 'attributes': [{'attribute_type_id': 'biolink:support_graphs',
   'value': 'SG:_uuid:1_biolink:treats_MONDO:0004979',
   'attribute_source': 'infores:answercoalesce'}]}

In [147]:
# CURIE-PHYSIO
print(f"{group2curie_aux_graph_edge}:===>")
for attributes in kg_edges[group2curie_aux_graph_edge]["attributes"]:
    sgs = attributes['value']
    sgedges = sorted(aux_graphs[sgs]["edges"])
    j = 0; path = 1
    while j < len(sgedges)-1:
        p1 = kg_edges[sgedges[j]]
        p2 = kg_edges[sgedges[j+1]]
        print(f'Path{path}: {kg_nodes[p2["subject"]]["name"]}-({p2["predicate"]})->{kg_nodes[p2["object"]]["name"]} ^ {kg_nodes[p1["subject"]]["name"]}-({p1["predicate"]})->{kg_nodes[p1["object"]]["name"]}')
        j+=2
        path+=1

uuid:1_biolink:treats_MONDO:0004979:===>
Path1: zafirlukast-(biolink:member_of)->uuid:1 ^ zafirlukast-(biolink:treats)->asthma
Path2: zileuton-(biolink:member_of)->uuid:1 ^ zileuton-(biolink:treats)->asthma
Path3: pemirolast-(biolink:member_of)->uuid:1 ^ pemirolast-(biolink:treats)->asthma
Path4: (-)-ephedrine-(biolink:member_of)->uuid:1 ^ (-)-ephedrine-(biolink:treats)->asthma
Path5: (R)-noradrenaline-(biolink:member_of)->uuid:1 ^ (R)-noradrenaline-(biolink:treats)->asthma
Path6: desonide-(biolink:member_of)->uuid:1 ^ desonide-(biolink:treats)->asthma
Path7: albuterol-(biolink:member_of)->uuid:1 ^ albuterol-(biolink:treats)->asthma
Path8: theophylline-(biolink:member_of)->uuid:1 ^ theophylline-(biolink:treats)->asthma
Path9: (R)-adrenaline-(biolink:member_of)->uuid:1 ^ (R)-adrenaline-(biolink:treats)->asthma
Path10: beclomethasone-(biolink:member_of)->uuid:1 ^ beclomethasone-(biolink:treats)->asthma
Path11: betamethasone-(biolink:member_of)->uuid:1 ^ betamethasone-(biolink:treats)->as